In [ ]:
import numpy as np
from keras.layers import Activation,Dense,Dropout
from keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from sklearn import datasets as skds
from keras.preprocessing.text import Tokenizer
import pickle
from keras.models import load_model
from pathlib import Path
import pandas as pd

In [ ]:
path_data = "/content/drive/MyDrive/text classification/bbc"

In [ ]:
files = skds.load_files(path_data,load_content= False)
label_index = files.target
label_names = files.target_names
labelled_files = files.filenames

data_tags = ["filename","category","news"]
i = 0
data_list = []
for file in labelled_files:
  data_list.append((file, label_names[label_index[i]],Path(file).read_text()))
  i = i+1

data = pd.DataFrame.from_records(data_list,columns = data_tags)



In [ ]:
'''
for notebook::
data_tags = ["filename","category","news"]
i = 0
data_list = []
for f in labelled_files:
  data_list.append(f,label_name[label_index[i]])
  b = open(f,'r')
  c = b.read()
  print(c)
  data_list.append((c,))
  data_list.append(f,label_names[label_index[i]])
  i = i+1

print(data_list)
'''

In [ ]:
data

,filename,category,news
0,/content/drive/MyDrive/text classification/bbc...,tech,Net regulation 'still possible'\n\nThe blurrin...
1,/content/drive/MyDrive/text classification/bbc...,business,VW considers opening Indian plant\n\nVolkswage...
2,/content/drive/MyDrive/text classification/bbc...,business,Court rejects $280bn tobacco case\n\nA US gove...
3,/content/drive/MyDrive/text classification/bbc...,business,AstraZeneca hit by drug failure\n\nShares in A...
4,/content/drive/MyDrive/text classification/bbc...,business,J&J agrees $25bn Guidant deal\n\nPharmaceutica...
...,...,...,...
1515,/content/drive/MyDrive/text classification/bbc...,politics,Labour chooses Manchester\n\nThe Labour Party ...
1516,/content/drive/MyDrive/text classification/bbc...,politics,Report attacks defence spending\n\nThe Ministr...
1517,/content/drive/MyDrive/text classification/bbc...,tech,"Rich pickings for hi-tech thieves\n\nViruses, ..."
1518,/content/drive/MyDrive/text classification/bbc...,entertainment,McCririck out of Big Brother show\n\nRacing pu...


In [ ]:
df = data
data_len = len(df)
train_data = int(data_len*0.8)
data_len

1520

In [ ]:
train_tag = df['category'][:train_data]
train_post = df['news'][:train_data]  # tokanize garnu agadi ko x_train ho yo

test_tag = df['category'][train_data:]
test_post = df['news'][train_data:]

In [ ]:
import numpy as np
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer

In [ ]:
num_labels = 4       # categories haru mathi herera
vocab_size = 1500    # (1 sec)ekchoti ma kati ota text data lai vector ko format ma pathauney
                     # news ma kati words cha tesma depend garcha 
                     # i.e 0 ma tech category ma kati words cha tah
### vocab_size ---> euta category ko news ma number of words kati cha bhanne?
batch_size = 100
# mathi ko data haru guess garera rakhne ho pachi tuning garera milauna sakincha


### define Tokenizer with vocab size  ###
# kun word chai kun class sanga related cha bhanne
# i.e bank ma jada line milauna token diyeko jasto
# NN ley queue lai manage garna token use garcha

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_post) # tokanize chai content(text) lai garne

# tokanize chai post(content) lai garne
x_train = tokenizer.texts_to_matrix(train_post,mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_post,mode ='tfidf')

encoder = LabelBinarizer() # CNN ma sequential layer use gareko jasto 
encoder.fit(train_tag)  # encode chai tag(i.e category hamro mathi ko case ma) lai garne

# encode chai tag (i.e category ) lai garne
y_train = encoder.transform(train_tag)
y_test = encoder.transform(test_tag) 

In [ ]:
# NN ko kaam start bhayo aba
model = Sequential()
model.add(Dense(512,input_shape = (vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               768512    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs = 10,validation_split = 0.1)

Epoch 1/10
35/35 [==============================] - 1s 14ms/step - loss: 0.8885 - accuracy: 0.6576 - val_loss: 0.1522 - val_accuracy: 0.9508
Epoch 2/10
35/35 [==============================] - 0s 13ms/step - loss: 0.0810 - accuracy: 0.9679 - val_loss: 0.0966 - val_accuracy: 0.9672
Epoch 3/10
35/35 [==============================] - 0s 9ms/step - loss: 0.0107 - accuracy: 0.9977 - val_loss: 0.0671 - val_accuracy: 0.9754
Epoch 4/10
35/35 [==============================] - 0s 9ms/step - loss: 0.0057 - accuracy: 0.9978 - val_loss: 0.0935 - val_accuracy: 0.9672
Epoch 5/10
35/35 [==============================] - 0s 9ms/step - loss: 0.0102 - accuracy: 0.9986 - val_loss: 0.0736 - val_accuracy: 0.9836
Epoch 6/10
35/35 [==============================] - 0s 9ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0669 - val_accuracy: 0.9754
Epoch 7/10
35/35 [==============================] - 0s 9ms/step - loss: 0.0053 - accuracy: 0.9992 - val_loss: 0.0674 - val_accuracy: 0.9754
Epoch 8/10
35/35 [

In [ ]:
# saving the model

import pickle
model.save('/content/drive/MyDrive/text classification/bbc2.h5')
with open('tokenizer.pickle','wb') as f:
  pickle.dump(tokenizer,f,protocol = pickle.HIGHEST_PROTOCOL)
# pickle chai serialization ko lagi chahincha

In [ ]:
# loading the model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/text classification/bbc2.h5')

# tokenizer = Tokenizer()
# aba read mode ma kholne
# model testing ko lagi
with open('tokenizer.pickle','rb') as f:
  tokenizers = pickle.load(f)

In [ ]:
encoder.classes_

array(['business', 'entertainment', 'politics', 'tech'], dtype='<U13')

In [ ]:
labels = np.array(['business', 'entertainment', 'politics', 'tech'])

In [ ]:
b = open('news2.txt','w')
b.write("A mobile phone has become more than just a medium to call somebody. It has become something more. Its multiple uses have made it a necessity for most of us, especially for those who use social media. And, if you are on any social media platform, you are bound to post stuff on it, including pictures.")
b.close()

In [ ]:
b = open('news2.txt','r')
print(b.read())

A mobile phone has become more than just a medium to call somebody. It has become something more. Its multiple uses have made it a necessity for most of us, especially for those who use social media. And, if you are on any social media platform, you are bound to post stuff on it, including pictures.


In [ ]:
test_files = ['news2.txt']

In [ ]:
from pathlib import Path
import pandas as pd
x_data = []
for t_f in test_files:
  t_f_data = Path(t_f).read_text()
  x_data.append(t_f_data)

x_data_series = pd.Series(x_data)
x_tokenized = tokenizer.texts_to_matrix(x_data_series, mode = 'tfidf')

In [ ]:
# for prediction
for x_t in x_tokenized:
  prediction = model.predict(np.array([x_t]))
  print(prediction)
  
  predicted_label = np.argmax(prediction)
  print(predicted_label)

  predicted_label = labels[np.argmax(prediction)]
  print(predicted_label)

[[0.07308846 0.11211583 0.03174089 0.7830548 ]]
3
tech


In [ ]:
for i in range(10):
  prediction = model.predict(np.array([x_test[i]]))
  predicted_label = labels[np.argmax(prediction)]
  print(prediction)
  print(predicted_label)
  print('Actual Label:',test_tag.iloc[i])

[[3.3026340e-13 1.1022274e-13 2.8401281e-09 1.0000000e+00]]
tech
Actual Label: tech
[[1.1319707e-07 9.9999928e-01 1.3012733e-07 5.2237465e-07]]
entertainment
Actual Label: entertainment
[[5.0112541e-04 9.9837214e-01 2.8943649e-04 8.3729101e-04]]
entertainment
Actual Label: entertainment
[[9.8025060e-01 1.1146893e-04 1.9548396e-02 8.9559777e-05]]
business
Actual Label: business
[[9.9681002e-01 1.7046756e-03 1.2788232e-03 2.0635815e-04]]
business
Actual Label: business
[[6.2400252e-11 4.7671903e-13 1.0000000e+00 7.0731610e-13]]
politics
Actual Label: politics
[[7.8792922e-10 9.9999988e-01 6.3367288e-08 5.8553876e-08]]
entertainment
Actual Label: entertainment
[[8.36081563e-06 1.38216834e-07 9.99991417e-01 1.05163856e-07]]
politics
Actual Label: politics
[[9.2123926e-08 2.9234873e-02 9.7076076e-01 4.2655379e-06]]
politics
Actual Label: politics
[[1.75066824e-15 5.21046229e-16 1.00000000e+00 1.12992035e-14]]
politics
Actual Label: politics
